# Federated Learning Case Study

## 04. Federated Learning Example using CICIDS2017 (Improved dataset) with Flower and Tensorflow/Keras

In this notebook we use the Flower Federated Learning library (flower.dev) with Tensorflow/Keras to distribute the CICIDS2017 data across multiple clients.


### 1. Preparing the CICIDS dataset

In [1]:
import os
import pandas as pd
import numpy as np

# We load in the data generated from notebook two of this series
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [2]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(1679923, 79)
(419981, 79)
(1679923,)
(419981,)


### 2. Set up the Federated Learning system

In [4]:
from collections import OrderedDict
from typing import List, Tuple

In [5]:
NUM_OF_CLIENTS = 10
NUM_OF_ROUNDS = 5

In [6]:
# We use the sklearn function for Stratified KFold groups, to distribute a balanced set of examples for each client.
# Further experimentation will look at different splits of how the data is distributed amongst the clients.

import numpy as np
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=NUM_OF_CLIENTS, shuffle=True, random_state=42)
skf.get_n_splits(X_train, y_train)

fl_X_train = []
fl_y_train = []

for train_index, test_index in skf.split(X_train, y_train):
    fl_X_train.append(X_train[test_index])
    fl_y_train.append(y_train[test_index]) 


In [7]:
for i in range(len(fl_X_train)):
    print (i, ':', "X shape", fl_X_train[0].shape, " Y shape:" , fl_y_train[0].shape)

0 : X shape (167993, 79)  Y shape: (167993,)
1 : X shape (167993, 79)  Y shape: (167993,)
2 : X shape (167993, 79)  Y shape: (167993,)
3 : X shape (167993, 79)  Y shape: (167993,)
4 : X shape (167993, 79)  Y shape: (167993,)
5 : X shape (167993, 79)  Y shape: (167993,)
6 : X shape (167993, 79)  Y shape: (167993,)
7 : X shape (167993, 79)  Y shape: (167993,)
8 : X shape (167993, 79)  Y shape: (167993,)
9 : X shape (167993, 79)  Y shape: (167993,)


In [9]:
# Checking the uniqueness of the class labels
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17])

In [10]:
%%capture
!pip install flwr[simulation] tensorflow

In [11]:
import flwr as fl
import numpy as np
import tensorflow as tf

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


flwr 1.0.0
numpy 1.22.4
tf 2.10.0


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

In [13]:
class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=1, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, y_test)
        print ("Client ", self.cid, "Evaluating complete...", accuracy, loss)
        return loss, len(self.train_data), {"accuracy": accuracy}

In [14]:

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(256, activation='sigmoid'),
      Dense(128, activation='sigmoid'), 
      Dense(18, activation='sigmoid'),  
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)

In [15]:
from flwr.common import Metrics
from sklearn.metrics import log_loss

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    accuracy = sum(accuracies) / sum(examples)
    print ("Accuracy:", accuracy)
    return {"accuracy": sum(accuracies) / sum(examples)}



def get_evaluate_fn(server_model):
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    def evaluate(server_round, parameters, config):
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print ("Server Evaluating...")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        print ("Server Evaluating complete...", accuracy, loss)
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(256, activation='sigmoid'),
      Dense(128, activation='sigmoid'), 
      Dense(18, activation='sigmoid'),  
    ])
server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])




# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

INFO flower 2022-10-28 22:52:10,924 | app.py:142 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flower 2022-10-28 22:52:14,760 | app.py:176 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 2791659110.0, 'memory': 5583318222.0, 'CPU': 8.0}
INFO flower 2022-10-28 22:52:14,761 | server.py:86 | Initializing global parameters
INFO flower 2022-10-28 22:52:14,761 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2022-10-28 22:52:18,525 | server.py:274 | Received initial parameters from one random client
INFO flower 2022-10-28 22:52:18,526 | server.py:88 | Evaluating initial parameters
INFO flower 2022-10-28 22:52:18,527 | server.py:101 | FL starting
DEBUG flower 2022-10-28 22:52:18,528 | server.py:215 | fit_round 1: strategy sampled 10 clients (out of 10)


(launch_and_get_parameters pid=26676) Client ID: 2
(launch_and_fit pid=26676) Client ID: 2
(launch_and_fit pid=26676) Client  2 Training...
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)    1/5250 [..............................] - ETA: 58:03 - loss: 2.4684 - accuracy: 0.0000e+00
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)   24/5250 [..............................] - ETA: 11s - loss: 1.0109 - accuracy: 0.7773      
(launch_and_fit pid=26676)   50/5250 [..............................] - ETA: 10s - loss: 0.8185 - accuracy: 0.7906
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)   76/5250 [..............................] - ETA: 10s - loss: 0.7011 - accuracy: 0.8166
(launch_and_fit pid=26676)  102/5250 [..............................] - ETA: 10s - loss: 0.6166 - accuracy: 0.8300
(launch_and_fit pid=26676)  126/5250 [..............................] - ETA: 10s - loss: 0.5663 - accuracy: 0.8413
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)  149/5250 [........

DEBUG flower 2022-10-28 22:54:27,722 | server.py:229 | fit_round 1 received 10 results and 0 failures
WARNING flower 2022-10-28 22:54:27,742 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-10-28 22:54:27,743 | server.py:165 | evaluate_round 1: strategy sampled 5 clients (out of 10)


(launch_and_fit pid=26676) 
(launch_and_fit pid=26676) 5138/5250 [============================>.] - ETA: 0s - loss: 0.0945 - accuracy: 0.9731
(launch_and_fit pid=26676) 5170/5250 [============================>.] - ETA: 0s - loss: 0.0944 - accuracy: 0.9731
(launch_and_fit pid=26676) 5202/5250 [============================>.] - ETA: 0s - loss: 0.0941 - accuracy: 0.9732
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676) 5229/5250 [============================>.] - ETA: 0s - loss: 0.0938 - accuracy: 0.9732
(launch_and_fit pid=26676) 5250/5250 [==============================] - 10s 2ms/step - loss: 0.0936 - accuracy: 0.9733
(launch_and_fit pid=26676) Client  5 Training complete...
(launch_and_evaluate pid=26676) Client ID: 1
(launch_and_evaluate pid=26676) Client  1 Evaluating...
(launch_and_evaluate pid=26676) Client ID: 9
(launch_and_evaluate pid=26676) Client  9 Evaluating...
(launch_and_evaluate pid=26676) Client ID: 6
(launch_and_evaluate pid=26676) Client  6 Evaluating...
(launch_

DEBUG flower 2022-10-28 22:54:41,979 | server.py:179 | evaluate_round 1 received 0 results and 5 failures
DEBUG flower 2022-10-28 22:54:41,980 | server.py:215 | fit_round 2: strategy sampled 10 clients (out of 10)


...

(launch_and_evaluate pid=26676) Client ID: 6
(launch_and_evaluate pid=26676) Client  6 Evaluating...
(launch_and_fit pid=26676) Client ID: 4
(launch_and_fit pid=26676) Client  4 Training...
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)    1/5250 [..............................] - ETA: 51:24 - loss: 0.0055 - accuracy: 1.0000
(launch_and_fit pid=26676)   23/5250 [..............................] - ETA: 12s - loss: 0.0206 - accuracy: 0.9918  
(launch_and_fit pid=26676)   46/5250 [..............................] - ETA: 11s - loss: 0.0226 - accuracy: 0.9905
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)   69/5250 [..............................] - ETA: 11s - loss: 0.0206 - accuracy: 0.9909
(launch_and_fit pid=26676)   87/5250 [..............................] - ETA: 12s - loss: 0.0214 - accuracy: 0.9910
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676)  105/5250 [..............................] - ETA: 12s - loss: 0.0224 - accuracy: 0.9911
(launch_and_fit pid=26676)  125/

DEBUG flower 2022-10-28 23:05:17,968 | server.py:229 | fit_round 5 received 10 results and 0 failures
DEBUG flower 2022-10-28 23:05:17,994 | server.py:165 | evaluate_round 5: strategy sampled 5 clients (out of 10)


(launch_and_fit pid=26676) 
(launch_and_fit pid=26676) 5160/5250 [============================>.] - ETA: 0s - loss: 0.0235 - accuracy: 0.9914
(launch_and_fit pid=26676) 5190/5250 [============================>.] - ETA: 0s - loss: 0.0235 - accuracy: 0.9914
(launch_and_fit pid=26676) 
(launch_and_fit pid=26676) 5217/5250 [============================>.] - ETA: 0s - loss: 0.0235 - accuracy: 0.9914
(launch_and_fit pid=26676) 5246/5250 [============================>.] - ETA: 0s - loss: 0.0235 - accuracy: 0.9914
(launch_and_fit pid=26676) 5250/5250 [==============================] - 11s 2ms/step - loss: 0.0235 - accuracy: 0.9914
(launch_and_fit pid=26676) Client  5 Training complete...
(launch_and_evaluate pid=26676) Client ID: 4
(launch_and_evaluate pid=26676) Client  4 Evaluating...
(launch_and_evaluate pid=26676) Client ID: 3
(launch_and_evaluate pid=26676) Client  3 Evaluating...
(launch_and_evaluate pid=26676) Client ID: 2
(launch_and_evaluate pid=26676) Client  2 Evaluating...
(launch_

DEBUG flower 2022-10-28 23:05:31,873 | server.py:179 | evaluate_round 5 received 0 results and 5 failures
INFO flower 2022-10-28 23:05:31,874 | server.py:144 | FL finished in 793.3452113000001
INFO flower 2022-10-28 23:05:31,877 | app.py:180 | app_fit: losses_distributed []
INFO flower 2022-10-28 23:05:31,879 | app.py:181 | app_fit: metrics_distributed {}
INFO flower 2022-10-28 23:05:31,879 | app.py:182 | app_fit: losses_centralized []
INFO flower 2022-10-28 23:05:31,880 | app.py:183 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=26676) Client ID: 9
(launch_and_evaluate pid=26676) Client  9 Evaluating...
